# updated 

In [1]:
import sys
sys.path.insert(1, '/projects/home/mmasood1/TG GATE/active_learning/')

import os
import pandas as pd
import numpy as np

import torch
import deepchem as dc
from pytorch_lightning import seed_everything

import wandb
os.environ["WANDB_SILENT"] = "true"
wandb.login(key = "27edf9c66b032c03f72d30e923276b93aa736429")

import torch.multiprocessing as mp
import gc
torch.multiprocessing.set_sharing_strategy('file_system')

from utils.data_utils import scafoldsplit_train_test, convert_to_dataframe, get_initial_set_with_main_and_aux_samples, drop_unwanted_tasks
from utils.data_utils import convert_dataframe_to_dataloader, convert_df_to_dc_data_object

from utils.utils import wandb_init_model, compute_binary_classification_metrics_MT, active_learning_loop
from utils.model_utils import get_pred_with_uncertainities
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

# In[2]:


seed_list = [1,2,3,4,5]
gpu = 0
task = "Y"
for seed in seed_list:
    config = {
            "seed": seed,
            # directories
            "project_name": "EPIG_Vanilla_BERT_DILI",
            
            ### Vanilla EPIG ###
            "metadata_dir": '/projects/home/mmasood1/trained_model_predictions/DILI/Frozen_BERT/Main_Aux_tasks_setting/1_main_0_aux/',
            "target_dir": f"/projects/home/mmasood1/arslan_data_repository/DILI/seed_{seed}/normalized_data/filtered_data/",        
            "pos_weights": f"/projects/home/mmasood1/arslan_data_repository/DILI/seed_{seed}/normalized_data/filtered_data/class_weight_seed_{seed}.csv",
            
            # data
            "features_type" :"DILI_dataset",
            "FP_size" : 1024,
            "train_frac": 0.8,

            # architechture
            "input_dim": 768,
            "hidden_dim": 128,
            "depth" : 1,
            "dropout_p": 0.3,
            "BatchNorm1d": True,
            "use_skip_connection": True,
        
            # training
            "optim": 'Adam',#SGD
            "lr_schedulers": "CosineAnnealingLR",
            "lr": 1e-3,
            "l2_lambda": 0.0,
            "optm_l2_lambda": 1e-2,
            "epochs": 2,
            "compute_metric_after_n_epochs": 5,
            "batch_size": 16,
            "EarlyStopping": False, 
            "pretrained_model": False,
            
            # loss
            "missing" : 'nan',
            "alpha": 0.0,
            "beta": 0.0,
            "gamma":0.0,

            "gpu": [gpu],
            "accelerator": "gpu",
            "return_trainer": True, 
            "save_predicitons" : True,
            "Final_model": False,

            # active learning
            "num_forward_passes": 20,
            "num_itterations": 200,
            "sampling_strategy": "EPIG_MT",
            "n_query":1,
            "main_task_samples": 100
        }
    if config["gpu"] == [0]:
        config["device"]  = torch.device("cuda:0")
    if config["gpu"] == [1]:
        config["device"]  = torch.device("cuda:1")
    if config["gpu"] == [2]:
        config["device"]  = torch.device("cuda:2")
    if config["gpu"] == [3]:
        config["device"]  = torch.device("cuda:3")
    # get targets information

    data = pd.read_csv(config["target_dir"] + f"train_filtered_seed_{seed}.csv")
    all_tasks = list(data.loc[:, "Y"].name)
    config["all_tasks"] = all_tasks
    config["main_task"] = all_tasks
    config["aux_task"] = None
    config["main_task_index"] = 0
    config["aux_task_index"] = None

    target_names = config["main_task"]
    config["project_name"] = config["project_name"] +"_"+ config["main_task"][0]

    config["num_of_tasks"] = len(target_names)
    config["selected_tasks"] = target_names

    config["sample_only_from_aux"] = False
    config["loss_type"] = "BCE" #"Focal_loss",# "BCE","Focal_loss_v2"


    # In[3]:


    # train, test, val, pool set
    train_set = pd.read_csv(config["target_dir"]+ f"train_filtered_seed_{seed}.csv")
    val_set = pd.read_csv(config["target_dir"]+ f"valid_filtered_seed_{seed}.csv")
    test_set = pd.read_csv(config["target_dir"]+ f"test_filtered_seed_{seed}.csv")

    # get dc_data_object
    train_set = convert_df_to_dc_data_object(train_set, config)
    val_set = convert_df_to_dc_data_object(val_set, config)
    test_set = convert_df_to_dc_data_object(test_set, config)

    initial_set, pool_set = get_initial_set_with_main_and_aux_samples(train_set, config)

    print("train_set", sorted(np.nansum(train_set.y, axis=0)))
    print("test_set", sorted(np.nansum(test_set.y, axis=0)))
    print("pool_set", sorted(np.nansum(pool_set.y, axis=0)))
    print("val_set", sorted(np.nansum(val_set.y, axis=0)))
    print("initial_set", sorted(np.nansum(initial_set.y, axis=0)))


    # In[4]:


    # Who cares about deepchem data_object, trash it
    initial_set = convert_to_dataframe(initial_set, config["selected_tasks"])
    val_set = convert_to_dataframe(val_set, config["selected_tasks"])
    pool_set = convert_to_dataframe(pool_set, config["selected_tasks"])
    test_set = convert_to_dataframe(test_set, config["selected_tasks"])

    if config["sample_only_from_aux"]:
        pool_set.loc[:,config["main_task"]] = np.nan

    # make dir
    t_names = config["metadata_dir"] + config["sampling_strategy"] + "/" + config["main_task"][0]
    query_set_dir = t_names +"/query_set/"
    result_dir = t_names +"/Results/"
    config["model_weights_dir"] = t_names +"/model_weights/"
    os.makedirs(query_set_dir, exist_ok = True)
    os.makedirs(result_dir, exist_ok = True)
    os.makedirs(config["model_weights_dir"], exist_ok = True)

    file_path = query_set_dir + "initial_set.csv"
    with open(file_path, 'w') as file:
        initial_set.to_csv(file, index = False)

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


train_set [173.0]
test_set [49.0]
pool_set [123.0]
val_set [11.0]
initial_set [50.0]
train_set [156.0]
test_set [49.0]
pool_set [106.0]
val_set [28.0]
initial_set [50.0]
train_set [158.0]
test_set [49.0]
pool_set [108.0]
val_set [26.0]
initial_set [50.0]
train_set [158.0]
test_set [49.0]
pool_set [108.0]
val_set [26.0]
initial_set [50.0]
train_set [163.0]
test_set [49.0]
pool_set [113.0]
val_set [21.0]
initial_set [50.0]


In [2]:
itteration = 1
from utils.models import Vanilla_MLP_classifier
seed_everything(seed = config["seed"])
config["itteration"] = itteration
config["model_name"] = rf'itteration_{config["itteration"]}_s{config["seed"]}_alpha_{config["alpha"]}_gamma_{config["gamma"]}_loss_type_{config["loss_type"]}_λ{config["optm_l2_lambda"]}'

Global seed set to 5


In [3]:
train_set_main_active,train_set_main_inactive =  (initial_set[config["main_task"]] == 1).sum().values, (initial_set[config["main_task"]] == 0).sum().values
if config["aux_task"] != None:
    train_set_aux_active,train_set_aux_inactive = (initial_set[config["aux_task"]] == 1).sum().sum(), (initial_set[config["aux_task"]] == 0).sum().sum()
else:
    train_set_aux_active,train_set_aux_inactive = np.array([0]),np.array([0])    
train_set_main_total = train_set_main_active + train_set_main_inactive
train_set_aux_total = train_set_aux_active + train_set_aux_inactive

# get dataloaders
train_dataloader = convert_dataframe_to_dataloader(dataframe= initial_set, config = config, shuffle= True, drop_last = True)
val_dataloader = convert_dataframe_to_dataloader(dataframe= val_set, config = config, shuffle= False, drop_last = False)
test_dataloader = convert_dataframe_to_dataloader(dataframe= test_set, config = config, shuffle= False, drop_last = False)
pool_dataloader = convert_dataframe_to_dataloader(dataframe= pool_set, config = config, shuffle= False, drop_last = False)

# Train model
config["training_steps"] = len(train_dataloader)
trained_model, run, trainer = wandb_init_model(model = Vanilla_MLP_classifier, 
                                                        train_dataloader = train_dataloader,
                                                        val_dataloader =val_dataloader,
                                                        config = config, 
                                                        model_type = 'MLP')

train_set_main_total = train_set_main_active + train_set_main_inactive
train_set_aux_total = train_set_aux_active + train_set_aux_inactive
wandb.log({"train_set_main_total":train_set_main_total.item(),
        "train_set_main_active":train_set_main_active.item(),
        "train_set_main_inactive":train_set_main_inactive.item()})

wandb.log({"train_set_aux_total":train_set_aux_total.item(),
        "train_set_aux_active":train_set_aux_active.item(),
        "train_set_aux_inactive":train_set_aux_inactive.item()})

wandb.finish()

After merging (100, 768)
After merging (47, 768)
After merging (94, 768)
After merging (221, 768)


/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /projects/home/mmasood1/trained_model_predictions/DILI/Frozen_BERT/Main_Aux_tasks_setting/1_main_0_aux/EPIG_MT/Y/model_weights/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                   | Type              | Params
--------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (6) is smal

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [9]:
# Evaluation
trained_model = trained_model.eval()
targets, pred_mean, pred_var, all_pred = get_pred_with_uncertainities(test_dataloader, trained_model,
                                                    n_samples=test_set.shape[0],
                                                    n_classes=config["num_of_tasks"],
                                                    cal_uncert=True,
                                                    num_forward_passes= 2)

In [7]:
pred_mean

array([[0.29079013],
       [0.38216284],
       [0.30787875],
       [0.6211236 ],
       [0.23830371],
       [0.22172946],
       [0.4160882 ],
       [0.73192789],
       [0.69951222],
       [0.17104351],
       [0.28111243],
       [0.93334213],
       [0.72957969],
       [0.71890509],
       [0.71121417],
       [0.15876704],
       [0.29403858],
       [0.68864489],
       [0.54242337],
       [0.66642582],
       [0.64567175],
       [0.75291109],
       [0.78784095],
       [0.24423802],
       [0.243955  ],
       [0.51924188],
       [0.75937826],
       [0.61018602],
       [0.64913057],
       [0.11779211],
       [0.24692854],
       [0.67545281],
       [0.39679827],
       [0.79698812],
       [0.82978136],
       [0.17474522],
       [0.18181082],
       [0.19760751],
       [0.5029156 ],
       [0.61428587],
       [0.55893951],
       [0.24114593],
       [0.74800767],
       [0.38357812],
       [0.71021907],
       [0.28773498],
       [0.55709805],
       [0.134

In [9]:
from scipy.special import expit
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()


def get_pred_with_uncertainities(dataloader, model, n_samples, n_classes, cal_uncert=False, num_forward_passes=1, device = None):

    dropout_predictions = []
    for i in range(num_forward_passes):
        torch.manual_seed(i)
        model = model.eval()
        model = model.to(device)
        if cal_uncert == True:
            enable_dropout(model)
        preds, targets = [], []
        for batch in dataloader:
            batch_x, batch_targets = batch
            with torch.no_grad():
                batch_preds = model(batch_x.to(device))
                batch_preds = batch_preds.cpu().detach().tolist()
                preds.extend(batch_preds)
                targets.extend(batch_targets.cpu().detach().tolist())
                 
        preds = expit(np.array(preds))
        targets = np.array(targets)
        dropout_predictions.append(preds.reshape(-1, n_samples, n_classes))

    dropout_predictions = np.concatenate(dropout_predictions, axis=0)
        
    pred_mean = np.mean(dropout_predictions, axis=0)
    pred_var = np.var(dropout_predictions, axis=0)

    return targets, pred_mean, pred_var, dropout_predictions

In [11]:
from utils.model_utils import BALD_acquisition_function, EPIG_MT_acquisition_function, get_top_indices
from utils.data_utils import get_random_query_set, get_query_set, update_training_set, remove_queried_index_from_pool_set

def active_learning_loop(trained_model,
                         pool_dataloader, 
                         initial_set,
                         pool_set, 
                         config,
                         query_set = None,
                         test_dataloader = None,
                         test_set = None):
    if query_set is None:
        ####### Uncertainity estmation ################
        if config["sampling_strategy"] == "BALD":
            _, _, _, all_pred = get_pred_with_uncertainities(pool_dataloader, trained_model,
                                                            n_samples=pool_set.shape[0],
                                                            n_classes=config["num_of_tasks"],
                                                            cal_uncert=True,
                                                            num_forward_passes=config["num_forward_passes"],
                                                            device = config["device"])
            acquisition = BALD_acquisition_function(all_pred)
            
            # We should not query with missing labels, so hide it
            nan_mask = ~np.isnan(pool_set[config["selected_tasks"]].values)
            acquisition = acquisition * nan_mask

            print(np.max(acquisition))

            # Get location of TopGuns
            top_indices = get_top_indices(acquisition, config["n_query"])
            query_set = get_query_set(pool_set, top_indices)

        if config["sampling_strategy"] == "EPIG_MT":
            print("######### EPIG SAMPLING ############")
            _, _, _, pool_pred = get_pred_with_uncertainities(pool_dataloader, trained_model,
                                                            n_samples=pool_set.shape[0],
                                                            n_classes=config["num_of_tasks"],
                                                            cal_uncert=True,
                                                            num_forward_passes=config["num_forward_passes"],
                                                            device = config["device"])

            pool_pred = torch.from_numpy(pool_pred)
            print("step 1")
            _,_, _, test_pred = get_pred_with_uncertainities(test_dataloader, trained_model,
                                                            n_samples=test_set.shape[0],
                                                            n_classes=config["num_of_tasks"],
                                                            cal_uncert=True,
                                                            num_forward_passes=config["num_forward_passes"],
                                                            device = config["device"])
            test_pred = torch.from_numpy(test_pred)
            print("step 2")
            # we are intereseted in one task
            test_pred_main_task = test_pred[:,:,config["main_task_index"]]
            test_pred_main_task = torch.unsqueeze(test_pred_main_task, dim = 2)
            
            acquisition = EPIG_MT_acquisition_function(pool_pred, test_pred_main_task)
            acquisition = acquisition.detach().numpy()
            
            # We should not query with missing labels, so hide it
            nan_mask = ~np.isnan(pool_set[config["selected_tasks"]].values)
            acquisition = acquisition * nan_mask
            print(np.max(acquisition))

            # Get location of TopGuns
            top_indices = get_top_indices(acquisition, config["n_query"])
            query_set = get_query_set(pool_set, top_indices)

        if config["sampling_strategy"] == "uniform":
            query_set = get_random_query_set(pool_set, config)  
    else:
        print("No sample taken")      
    ##########  updated dataset ##################
    updated_training_set = update_training_set(initial_set, query_set)
    updated_poolset = remove_queried_index_from_pool_set(pool_set, query_set, config)

    initial_counts = initial_set.iloc[:, 1:].count().sum()
    query_counts = query_set.iloc[:, 1:].count().sum()
    updated_training_counts = updated_training_set.iloc[:, 1:].count().sum()

    initial_poolset_counts = pool_set.iloc[:, 1:].count().sum()
    updated_poolset_counts = updated_poolset.iloc[:, 1:].count().sum()

    print(
            "initial_counts", initial_counts, 
            "initial_poolset_count", initial_poolset_counts, 
            "query_counts", query_counts, 
            "updated_training_counts", updated_training_counts, 
            "updated_poolset_counts", updated_poolset_counts)
    
    # Use an assertion to check if the counts are equal
    assert updated_training_counts == initial_counts + query_counts, "Training_count,Queryset counts are not equal"
    assert updated_poolset_counts == initial_poolset_counts - query_counts, "Poolset, Queryset count are not equal"
    return query_set, updated_training_set, updated_poolset

In [12]:
query_set, updated_training_set, updated_poolset = active_learning_loop(trained_model,
                                                                                pool_dataloader, 
                                                                                initial_set,
                                                                                pool_set, 
                                                                                config)

######### EPIG SAMPLING ############
step 1


AttributeError: 'NoneType' object has no attribute 'shape'